# Testing C-integration

Here’s the gist of it:

```template=np.empty(1,dtype=np.dtype(D_gal,align=True))
gals=np.full(n_gal,template)```

The key here being the `align=True` in the generation of the galaxy template.

```C_lib.make_stars.argtypes=(np.ctypeslib.ndpointer(gals.dtype),ctypes.c_double)```

The `ndpointer` function generates a pointer to the numpy datatype.

```C_lib.make_stars(gals[i_gal:i_gal+1],ctypes.c_double(i_gal))```

The only weird thing here is that `gals[i_gal]` is not a numpy array and throws type errors.  However, `gals[i_gal:i_gal+1]` is a view to the same object that is a numpy array.

In [ ]:
import ctypes
import numpy as np

In [ ]:
print('Python:')

# Create some dummy galaxy data for use in testing
n_gal=10

# Create the dtype that we will need to store galaxy properties.
D_gal=[
   ('gal_gid',ctypes.c_int),      # The unique identifier for this galaxy
   ('b_exists',ctypes.c_bool),
   ('mass_stars',ctypes.c_double)
]

NDI=-2147483648
template=np.empty(1,dtype=np.dtype(D_gal,align=True))
template['gal_gid']=NDI
template['b_exists']=False
template['mass_stars']=0.

gals=np.full(n_gal,template)
for i_gal in range(n_gal):
    gals['gal_gid'][i_gal]=i_gal
    gals['b_exists']=True
    gals['mass_stars']=0.
    
print(gals)
print(template.dtype.itemsize)

In [ ]:
# Python version of functions
def make_stars(gal,new_stars):
    gal['mass_stars']+=new_stars
    return None

def merge_gals(gal1,gal2):
    gal1['mass_stars']+=gal2['mass_stars']
    gal2['mass_stars']=0.
    gal2['b_exists']=False
    return None

In [ ]:
# Main program
for i_gal in range(n_gal):
    make_stars(gals[i_gal],float(i_gal))
for i_gal in range(0,n_gal,2):
    merge_gals(gals[i_gal],gals[i_gal+1])
print(gals)
print()

In [ ]:
print('ctypes:')

# Now set up use of C version of the program
import ctypes

# Note: adding ",flags='contiguous',aligned'"" to the ndpointer call seems to make no difference.
C_lib=ctypes.CDLL('lib_C.so')
C_lib.make_stars.argtypes=(np.ctypeslib.ndpointer(gals.dtype),ctypes.c_double)
C_lib.make_stars.restype=None
C_lib.merge_gals.argtypes=(np.ctypeslib.ndpointer(gals.dtype),np.ctypeslib.ndpointer(gals.dtype))
C_lib.merge_gals.restype=None


In [ ]:
# And initialise and run C version of the program

gals=np.full(n_gal,template)
for i_gal in range(n_gal):
    gals['gal_gid'][i_gal]=i_gal
    gals['b_exists']=True
    gals['mass_stars']=0.
print(gals)
    
for i_gal in range(n_gal):
    C_lib.make_stars(gals[i_gal:i_gal+1],ctypes.c_double(i_gal))
for i_gal in range(0,n_gal,2):
    C_lib.merge_gals(gals[i_gal:i_gal+1],gals[i_gal+1:i_gal+2])
    
print(gals)
print()

In [ ]:
print('Structures:')

# Can do it like this using an explict structure definition,
# but frankly that is a real pain and would be MUCH better to 
# get it to work using numpy structured arrays.

import ctypes

class gal_struct(ctypes.Structure):
    _fields_ = [
           ('gal_gid',ctypes.c_int),
           ('b_exists',ctypes.c_bool),
           ('mass_stars',ctypes.c_double),
           ('padding',ctypes.c_double)
    ]
gals = (gal_struct*10)()
for i_gal in range(n_gal):
    gals[i_gal].gal_gid=i_gal
    gals[i_gal].b_exists=True
    gals[i_gal].mass_stars=0.

C_lib=ctypes.CDLL('lib_C.so')
C_lib.make_stars.argtypes=(ctypes.POINTER(gal_struct),ctypes.c_double)
C_lib.make_stars.restype=None
C_lib.merge_gals.argtypes=(ctypes.POINTER(gal_struct),ctypes.POINTER(gal_struct))
C_lib.merge_gals.restype=None

In [ ]:
n_gal=10
for i_gal in range(n_gal):
    C_lib.make_stars(ctypes.byref(gals[i_gal]),float(i_gal))
for i_gal in range(0,n_gal,2):
    C_lib.merge_gals(gals[i_gal],gals[i_gal+1])
    
for i_gal in range(n_gal):
    print(gals[i_gal].gal_gid,gals[i_gal].b_exists,gals[i_gal].mass_stars)

### Creating a struct of parameters

Next we create a C struct of the attributes of the parameters C_parameters class instance.

In [ ]:
class C_parameters:
    def __init__(self,a=1,b=2.):
        self.a=a
        self.b=b
        self.c='three'
        self.d=False
    def set_d(self,true_or_false):
        self.d=true_or_false
        
parameters=C_parameters()
parameters.set_d(True)
parameters.e=2.718281828

dir(parameters)

In [ ]:
attributes=[a for a in dir(parameters) if not a.startswith('__') and not callable(getattr(parameters, a))]
f=open('parameters.h','w')
f.write('/* Runtime parameters (fixed throughout run). */\n\
\n\
#include <stdbool.h>\n\
\n\
struct {\n\
')
for a in attributes:
    value=eval('parameters.'+a)
    a_type=str(type(value)).split('\'')[1]
    print(a,a_type,value)
    if a_type == 'str':
        f.write('    char* '+a+'="'+str(value)+'";\n')
    elif a_type == 'bool':
        if value==True:
            f.write('    '+a_type+' '+a+'=true;\n')
        else:
            f.write('    '+a_type+' '+a+'=false;\n')
    else:
        f.write('    '+a_type+' '+a+'='+str(value)+';\n')
f.write('}; parameters;\n')
f.close()


In [ ]:
d={'one':1,'two':2,'three':3}

In [ ]:
'one' in d